# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 36

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
340,Lloris,Tottenham,53,19.111111,86,84.903405,4.5,4.442620,4.287210,4.460506,1.929751
411,Patrício,Wolves,53,36.250000,145,155.301197,4.0,4.284171,3.402546,3.745385,1.860925
471,Henderson,Sheffield Utd,52,34.130435,157,141.680761,4.6,4.151156,4.009932,4.145201,1.820402
168,Schmeichel,Leicester City,55,35.813953,154,147.008890,4.3,4.104794,3.449691,4.436802,1.750290
93,Pope,Burnley,51,36.222222,163,148.177295,4.5,4.090784,4.108761,4.296969,1.811430
235,de Gea,Manchester Utd,53,36.216216,134,141.869777,3.7,3.917300,4.269752,4.293618,1.701566
341,Gazzaniga,Tottenham,41,18.125000,58,69.128011,3.2,3.813959,NaN,1.653034,1.883580
189,Alisson,Liverpool,62,27.045455,119,103.026729,4.4,3.809392,3.145642,3.368351,1.529889
131,Guaita,Crystal Palace,49,33.250000,133,126.256094,4.0,3.797176,2.830783,3.447557,1.715389


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,77,36.296296,196,184.910959,5.4,5.094486,3.019200,3.510383,1.835925
181,Robertson,Liverpool,70,34.200000,171,160.637876,5.0,4.697014,5.423034,4.497258,1.775304
401,Doherty,Wolves,65,33.695652,155,158.084782,4.6,4.691548,3.439842,4.465548,1.840177
103,Alonso,Chelsea,61,15.892857,89,74.560523,5.6,4.691449,4.258637,4.260717,1.899512
105,Azpilicueta,Chelsea,61,33.783784,125,155.320688,3.7,4.597492,5.362069,5.219014,1.861470
183,van Dijk,Liverpool,65,35.744681,168,148.267821,4.7,4.147969,3.390344,3.519165,1.626967
297,Lundstram,Sheffield Utd,46,32.380952,136,133.694445,4.2,4.128799,2.121292,2.591918,1.925062
159,Pereira,Leicester City,62,27.954545,123,112.958467,4.4,4.040791,NaN,4.237695,1.622821
185,Matip,Liverpool,52,9.069767,39,35.850945,4.3,3.952797,NaN,4.079767,1.733416


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,125,31.971831,227,218.522116,7.1,6.834833,7.297781,6.794712,1.933183
618,Fernandes,Manchester Utd,91,11.931818,105,78.452311,8.8,6.575051,6.781902,6.804725,2.179608
215,De Bruyne,Manchester City,107,33.235294,226,206.276305,6.8,6.206544,6.011123,6.414709,1.897396
192,Mané,Liverpool,122,32.812500,210,190.750633,6.4,5.813353,5.269532,5.112684,1.664358
214,Sterling,Manchester City,119,30.862069,179,178.049625,5.8,5.769206,5.835396,6.076107,1.672409
239,Martial,Manchester Utd,85,30.000000,189,153.081812,6.3,5.102727,6.638190,6.529543,1.750221
342,Son,Tottenham,96,27.758621,161,139.770115,5.8,5.035197,5.028378,4.807558,1.625103
431,Pulisic,Chelsea,74,23.000000,115,111.536287,5.0,4.849404,5.099907,5.495451,1.782676
389,Antonio,West Ham,70,22.173913,102,106.390230,4.6,4.797991,8.073964,7.325073,1.813470


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,89,29.000000,174,166.450588,6.0,5.739675,6.424613,6.074265,1.923944
166,Vardy,Leicester City,98,32.857143,207,186.391765,6.3,5.672793,8.135898,6.848080,1.812107
409,Jiménez,Wolves,81,35.961538,187,180.072941,5.2,5.007376,5.174957,4.832004,1.759413
210,Agüero,Manchester City,117,24.000000,132,117.052941,5.5,4.877206,NaN,4.456819,1.425864
11,Aubameyang,Arsenal,109,34.000000,187,164.915294,5.5,4.850450,3.896053,4.516240,1.469159
313,Ings,Southampton,76,35.769231,186,166.197647,5.2,4.646386,5.438763,5.009357,1.685421
187,Firmino,Liverpool,93,35.750000,143,163.690588,4.0,4.578758,2.981720,2.994673,1.501434
460,Abraham,Chelsea,74,31.956522,147,142.692941,4.6,4.465221,2.409623,2.863661,1.641448
211,Jesus,Manchester City,98,32.380952,136,143.610588,4.2,4.435033,7.948163,5.942712,1.416719


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,125,31.971831,227,218.522116,7.1,6.834833,7.297781,6.794712,1.933183
618,Fernandes,Manchester Utd,91,11.931818,105,78.452311,8.8,6.575051,6.781902,6.804725,2.179608
215,De Bruyne,Manchester City,107,33.235294,226,206.276305,6.8,6.206544,6.011123,6.414709,1.897396
192,Mané,Liverpool,122,32.812500,210,190.750633,6.4,5.813353,5.269532,5.112684,1.664358
214,Sterling,Manchester City,119,30.862069,179,178.049625,5.8,5.769206,5.835396,6.076107,1.672409
233,Rashford,Manchester Utd,89,29.000000,174,166.450588,6.0,5.739675,6.424613,6.074265,1.923944
166,Vardy,Leicester City,98,32.857143,207,186.391765,6.3,5.672793,8.135898,6.848080,1.812107
239,Martial,Manchester Utd,85,30.000000,189,153.081812,6.3,5.102727,6.638190,6.529543,1.750221
182,Alexander-Arnold,Liverpool,77,36.296296,196,184.910959,5.4,5.094486,3.019200,3.510383,1.835925


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
389,Antonio,West Ham,70,22.173913,102,106.390230,4.6,4.797991,8.073964,7.325073,1.813470
166,Vardy,Leicester City,98,32.857143,207,186.391765,6.3,5.672793,8.135898,6.848080,1.812107
618,Fernandes,Manchester Utd,91,11.931818,105,78.452311,8.8,6.575051,6.781902,6.804725,2.179608
191,Salah,Liverpool,125,31.971831,227,218.522116,7.1,6.834833,7.297781,6.794712,1.933183
239,Martial,Manchester Utd,85,30.000000,189,153.081812,6.3,5.102727,6.638190,6.529543,1.750221
113,Willian,Chelsea,74,34.893617,164,156.930178,4.7,4.497389,6.035286,6.467671,1.653273
215,De Bruyne,Manchester City,107,33.235294,226,206.276305,6.8,6.206544,6.011123,6.414709,1.897396
214,Sterling,Manchester City,119,30.862069,179,178.049625,5.8,5.769206,5.835396,6.076107,1.672409
233,Rashford,Manchester Utd,89,29.000000,174,166.450588,6.0,5.739675,6.424613,6.074265,1.923944


In [11]:
metric = 'valuePoints metric'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,91,11.931818,105,78.452311,8.8,6.575051,6.781902,6.804725,2.179608
191,Salah,Liverpool,125,31.971831,227,218.522116,7.1,6.834833,7.297781,6.794712,1.933183
340,Lloris,Tottenham,53,19.111111,86,84.903405,4.5,4.442620,4.287210,4.460506,1.929751
297,Lundstram,Sheffield Utd,46,32.380952,136,133.694445,4.2,4.128799,2.121292,2.591918,1.925062
233,Rashford,Manchester Utd,89,29.000000,174,166.450588,6.0,5.739675,6.424613,6.074265,1.923944
103,Alonso,Chelsea,61,15.892857,89,74.560523,5.6,4.691449,4.258637,4.260717,1.899512
215,De Bruyne,Manchester City,107,33.235294,226,206.276305,6.8,6.206544,6.011123,6.414709,1.897396
341,Gazzaniga,Tottenham,41,18.125000,58,69.128011,3.2,3.813959,NaN,1.653034,1.883580
105,Azpilicueta,Chelsea,61,33.783784,125,155.320688,3.7,4.597492,5.362069,5.219014,1.861470
